In [24]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import pandas as pd 
import pymysql

In [35]:
# this chunk prepares the information to webscrape 
titles_df = pd.read_csv('first500.csv')
links = titles_df['Link'].tolist()
titles = titles_df['Title'].tolist()
years = titles_df['Year'].tolist()
ratings = titles_df['Rating'].tolist()
# un comment below line to run first two movies 
# parsing_through(titles, years, links, ratings, 2)

In [33]:
# give lists of all information from csv, plus limit of movies 
# eventually I want to change limit so that it reps page number 
def parsing_through(titles, years, links, ratings, limit):
    # for each movie 
    for i in range(limit):
        # getting movie specific information 
        title, year, link, rating = titles[i], years[i], links[i], ratings[i]
        review_list = collect_reviews(link) # list of reviews from collect function
        # creating list of rows 
        rows_to_insert = []
        # for each review collected, create new row
        for review in review_list:
            new_row = {'movie_title': title, 
                       'release_year': year, 'review': review,}
            rows_to_insert.append(new_row)
        # temporary data frame
        # still not sure how i want to split this, every movie? every page?
        temp_movies = pd.DataFrame(rows_to_insert)
        # right now i'm just printing to double check everything is correct
        print(temp_movies)
        

In [31]:
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)
# function to reveal the more button 
def more_review(count_li, p_tags):
    #p_tags = reviews because reviews catches all the p tags
    #using len(p_tags) as the number for which child of the p tag
    if len(p_tags) > 1:
        more = driver.find_element(By.CSS_SELECTOR, f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text > div > p:nth-child({len(p_tags)}) > a")
        #where the "more" button, always at the last paragraph so number would equal len(p_tags)
        #count_li corresponds to which of the 12 reviews
        more.click()
        wait = WebDriverWait(driver, 20)
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR,f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text > div > p:nth-child({len(p_tags)}) > a")))
        # most important line of code
        time.sleep(1)
        # get the review 
        review = driver.find_element(By.CSS_SELECTOR, f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text")
        #print the review 
    else:
        more = driver.find_element(By.CSS_SELECTOR, f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text > div > p > a")
        more.click()
        wait = WebDriverWait(driver, 20)
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR,f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text > div > p > a")))
        # most important line of code
        time.sleep(1)
        review = driver.find_element(By.CSS_SELECTOR, f"#content > div > div > section > section > ul > li:nth-child({count_li}) > div > div.body-text.-prose.collapsible-text")
        #print the review
    driver.close()
    return review

In [23]:
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)
# function that collects 20 pages of reviews for each movie, runs once for each link
def collect_reviews(link):
    movie_reviews = [] # creating list of movie reviews 
    # we eventually want to adjust range to fit everything, don't forget this 
    for url_count in range(1, 5): # iterate through each review page
        url = f"{link}reviews/by/activity/page/{url_count}/"
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        divs = soup.find_all("div", class_ = 'body-text -prose collapsible-text') # these contain the reviews
        driver.get(url)
        count_li = 1
        for i in divs: #iterate through reviews of first page,sorted by activit, 12 for each page 
            reviews = i.find_all("p")
            #find all p tags to be able to get over line breaks
            if reviews[0].text == "This review may contain spoilers. I can handle the truth.": 
                #reviews[0] because spoilers line is always the only line (paragraph 1)
                review = i.find("div", class_ = "hidden-spoilers expanded-text").text

            elif reviews[-1].text.find("… more") != -1:
                #reviews[-1] because "..." always last line (last paragraph)
                review = more_review(count_li, reviews).text
            else:
                #if neither spoilers or too long but still paragraph(works for single paragraph too)
                review = ""
                for j in reviews:
                    review += j.text + "\n"
            review = review.strip()
            # movies are appending in one big list 
            # ultimately we want to mass insert so we can save time 
            movie_reviews.append(review)
            count_li += 1
    driver.close()
    return(movie_reviews)
            